## Open farm and run simple diagnostic


## Load packages

In [1]:
!pip install odc-ui rasterstats

  Using cached odc_ui-0.2.0a3-py3-none-any.whl (15 kB)
  Using cached rasterstats-0.19.0-py3-none-any.whl (16 kB)
  Using cached jupyter_ui_poll-0.2.2-py2.py3-none-any.whl (9.0 kB)
  Using cached simplejson-3.19.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)


In [44]:
%%time
from grits import query_modis_items, humanbytes, get_field, get_lims, get_mms, query_l2a_items, xr_rasterize, calculate_indices


import time
start = time.time()

import warnings
warnings.filterwarnings('ignore')
# the basic
from geogif import gif
import rich.table
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

# for PC, stac, xarray
import odc.stac
import stackstac
from xrspatial import zonal_stats

# From DEA
import sys
sys.path.append('/home/jovyan/PlanetaryComputerExamples/grasspace/deafrica-sandbox-notebooks/Tools/deafrica_tools/')
from plotting import display_map, rgb, map_shapefile

# packages that have to be installed every runtime
import subprocess
import pkg_resources

required = {'rasterstats','odc-ui'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

CPU times: user 1.52 ms, sys: 859 µs, total: 2.38 ms
Wall time: 14.5 s


## Área de análise
Using function get_field for 

1. for fields in a farm - OK
2. for a group in a group of farms - OK
3. for specific fields in a farm - OK
4. ToDo - for an entire farm - the simplest case, as if the polygon was already given straight from a file

In [45]:
path = '/home/jovyan/PlanetaryComputerExamples/vetorial/FAZENDAS/'


#### Área: Fazenda Uniguiri
**Column parte** contém as regiões

In [46]:

field = gpd.read_file( path + 'fazenda_uniguiri.gpkg')

bbox, lat_range, lon_range = get_lims(field)

print(field.head())
#field.plot()

got bbox, lat_range, lon_range
   parte                                           geometry
0      1  MULTIPOLYGON (((-54.57278 -16.93966, -54.57686...
1      2  MULTIPOLYGON (((-54.57825 -16.95894, -54.57660...
2      3  MULTIPOLYGON (((-54.59494 -16.96052, -54.59700...
3      4  MULTIPOLYGON (((-54.65092 -16.95305, -54.65196...


#### Área: Iacanga - Cana

Grupo de talhoes dentro de todas fazendas de uma usina


In [47]:
%%time
# getting specific IDs
# example: talhoes especificos 
IDs = ['4140-034 NOSSA SENHORA APARECIDA', '380-028 SAO SEBASTIAO',
      '378-028 SAO SEBASTIAO', '375-028 SAO SEBASTIAO', '372-028 SAO SEBASTIAO', 
      '359-043 SANTA CANDIDA', '324-224 SAO ROQUE III', '323-224 SAO ROQUE III',
      '1658-326 BOA VISTA', '1656-333 ST CANTINHO DO CEU DOIS E TRES']
field = get_field( path + 'iacanga_22_23.gpkg',
                   ID = None,
                  column = 'TID',
                  multi_IDs=True,
                  IDs = IDs,
                  layer = 'talhoes')

bbox, lat_range, lon_range = get_lims(field)

#field.plot('TID')

got bbox, lat_range, lon_range
CPU times: user 1.36 s, sys: 39.3 ms, total: 1.4 s
Wall time: 1.39 s


### Display bbox study area


In [48]:
style={'opacity': 6, 'stroke': 2,'dashArray': '1', 'fillOpacity': 0.5}
column = 'AdMapKey'
map_shapefile(field,column,cmap='Set1', **style )

Label(value='')

Map(center=[-21.923873798892053, -49.25744693714713], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Get images

### get MODIS data
- LST
---
colocar as especificidades aqui

In [49]:
def query_modis_items2(bbox, 
                    datetime,
                    collection):

    '''
        Query MODIS items for a given bounding box withing a 
        datetime range 
        bbox.:tuple with coordinates of the 2 corners of a bounding box: it is retrieved by the 
                get_lims function
        collection.:str: collection.
        ... product? band?
    '''
    
    import pystac_client
    import planetary_computer

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )

    #query_params = {"eo:cloud_cover": {"lt": max_cloud_cover}}
    
    search = catalog.search(bbox=bbox,
                            collections=collections,
                            datetime=datetime
                            )
    
    items = search.item_collection()
    print(f' found {len(items)} items')

    return items


In [50]:
%%time
'''
    modis-11A1-061 - daily
    modis-11A2-061 - 8-day

'''


collections = ["modis-11A2-061"]
# daterange analysis
datetime = "2023-04-01/2023-10-08"

# a do grits n funciona
items = query_modis_items2(bbox, datetime, collections)

 found 51 items
CPU times: user 56.9 ms, sys: 4.57 ms, total: 61.5 ms
Wall time: 233 ms


In [51]:
t = rich.table.Table("Key", "Title")
for key, asset in items[0].assets.items():
    t.add_row(key, asset.title)
t

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Title                                                           ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ hdf              │ Source data containing all bands                                │
│ QC_Day           │ Quality control for daytime LST and emissivity                  │
│ Emis_31          │ Band 31 emissivity                                              │
│ Emis_32          │ Band 32 emissivity                                              │
│ QC_Night         │ Quality control for nighttime LST and emissivity                │
│ metadata         │ Federal Geographic Data Committee (FGDC) Metadata               │
│ LST_Day_1km      │ 8-day daytime 1km grid Landsurface Temperature                  │
│ Day_view_angl    │ Average view zenith angle of daytime Land-surface Temperature   │
│ Day_view_time    │ Average time of daytime Landsurface Temperature observation     │
│ LST_Night_1km    │ 8-day nighttime 1km grid Landsurface Temperature                │
│ Clear_sky_days   │ the days in clear-sky conditions and with validate LSTs         │
│ Night_view_angl  │ View zenith angle of nighttime Land-surface Temperature         │
│ Night_view_time  │ Average view zenith angle of nighttime Land-surface Temperature │
│ Clear_sky_nights │ the nights in clear-sky conditions and with validate LSTs       │
│ tilejson         │ TileJSON with default rendering                                 │
│ rendered_preview │ Rendered preview                                                │
└──────────────────┴─────────────────────────────────────────────────────────────────┘

### the datacube

In [52]:
%%time
data = odc.stac.load(
    items,
    crs="EPSG:3857",
    bbox=bbox,
    bands=["LST_Day_1km","LST_Night_1km","QC_Day","QC_Night"],
    resolution=500,
)

data

CPU times: user 11.9 s, sys: 11.3 s, total: 23.2 s
Wall time: 17 s


<xarray.Dataset>
Dimensions:        (y: 6, x: 8, time: 25)
Coordinates:
  * y              (y) float64 -2.501e+06 -2.502e+06 ... -2.503e+06 -2.504e+06
  * x              (x) float64 -5.485e+06 -5.485e+06 ... -5.482e+06 -5.482e+06
    spatial_ref    int32 3857
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
Data variables:
    LST_Day_1km    (time, y, x) uint16 15141 15141 15125 ... 15501 15438 15438
    LST_Night_1km  (time, y, x) uint16 14686 14686 14690 ... 14648 14655 14655
    QC_Day         (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    QC_Night       (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

the retrieval of a ~1000 images (500 m resolution) took 7 minutes. 537 kb. Estaria o problema na aquisição/montagem do cubo?

In [53]:
humanbytes(data.nbytes)

'7.34 KB'

## REPROJETRYING

In [54]:
import rioxarray as rxr
import rasterio as rio

In [14]:
#data.rio.write_crs('EPSG:3857', inplace=True)

<xarray.Dataset>
Dimensions:        (y: 6, x: 8, time: 25)
Coordinates:
  * y              (y) float64 -2.501e+06 -2.502e+06 ... -2.503e+06 -2.504e+06
  * x              (x) float64 -5.485e+06 -5.485e+06 ... -5.482e+06 -5.482e+06
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
    spatial_ref    int64 0
Data variables:
    LST_Day_1km    (time, y, x) uint16 15141 15141 15125 ... 15501 15438 15438
    LST_Night_1km  (time, y, x) uint16 14686 14686 14690 ... 14648 14655 14655
    QC_Day         (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    QC_Night       (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [55]:
datar = data.rio.reproject('EPSG:4326')
datar

<xarray.Dataset>
Dimensions:        (x: 8, y: 6, time: 25)
Coordinates:
  * x              (x) float64 -49.27 -49.27 -49.27 ... -49.25 -49.25 -49.24
  * y              (y) float64 -21.91 -21.92 -21.92 -21.93 -21.93 -21.94
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
    spatial_ref    int64 0
Data variables:
    LST_Day_1km    (time, y, x) uint16 15141 15141 15125 ... 15501 15438 15438
    LST_Night_1km  (time, y, x) uint16 14686 14686 14690 ... 14648 14655 14655
    QC_Day         (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    QC_Night       (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [16]:
datar.to_netcdf('/home/jovyan/PlanetaryComputerExamples/myout_nc/reproj4326.nc')

### Masks
- field
- values (trim)

In [35]:
rdata = xr.open_dataset('/home/jovyan/PlanetaryComputerExamples/myout_nc/reproj4326.nc')

In [21]:
rdata

<xarray.Dataset>
Dimensions:        (x: 8, y: 6, time: 25)
Coordinates:
  * x              (x) float64 -49.27 -49.27 -49.27 ... -49.25 -49.25 -49.24
  * y              (y) float64 -21.91 -21.92 -21.92 -21.93 -21.93 -21.94
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
Data variables:
    spatial_ref    int64 ...
    LST_Day_1km    (time, y, x) float32 ...
    LST_Night_1km  (time, y, x) float32 ...
    QC_Day         (time, y, x) float32 ...
    QC_Night       (time, y, x) float32 ...

In [40]:
datar2 = datar[[ "x", "y", "time","LST_Day_1km", "LST_Night_1km", "QC_Day", "QC_Night"]].copy()

In [38]:
datar2.rio.write_crs('EPSG:4326', inplace=True)

<xarray.Dataset>
Dimensions:        (y: 6, x: 8, time: 25)
Coordinates:
  * y              (y) float64 -21.91 -21.92 -21.92 -21.93 -21.93 -21.94
  * x              (x) float64 -49.27 -49.27 -49.27 ... -49.25 -49.25 -49.24
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
    spatial_ref    int64 0
Data variables:
    LST_Day_1km    (time, y, x) uint16 15141 15141 15125 ... 15501 15438 15438
    LST_Night_1km  (time, y, x) uint16 14686 14686 14690 ... 14648 14655 14655
    QC_Day         (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    QC_Night       (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [23]:
def xr_rasterize2(gdf,
                 da,
                 attribute_col=False,
                 crs=None,
                 transform=None,
                 name=None,
                 x_dim='x',
                 y_dim='y',
                 export_tiff=None,
                 verbose=False,
                 **rasterio_kwargs):    
    """
    Rasterizes a geopandas.GeoDataFrame into an xarray.DataArray.
    
    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        A geopandas.GeoDataFrame object containing the vector/shapefile
        data you want to rasterise.
    da : xarray.DataArray or xarray.Dataset
        The shape, coordinates, dimensions, and transform of this object 
        are used to build the rasterized shapefile. It effectively 
        provides a template. The attributes of this object are also 
        appended to the output xarray.DataArray.
    attribute_col : string, optional
        Name of the attribute column in the geodataframe that the pixels 
        in the raster will contain.  If set to False, output will be a 
        boolean array of 1's and 0's.
    crs : str, optional
        CRS metadata to add to the output xarray. e.g. 'epsg:3577'.
        The function will attempt get this info from the input 
        GeoDataFrame first.
    transform : affine.Affine object, optional
        An affine.Affine object (e.g. `from affine import Affine; 
        Affine(30.0, 0.0, 548040.0, 0.0, -30.0, "6886890.0) giving the 
        affine transformation used to convert raster coordinates 
        (e.g. [0, 0]) to geographic coordinates. If none is provided, 
        the function will attempt to obtain an affine transformation 
        from the xarray object (e.g. either at `da.transform` or
        `da.geobox.transform`).
    x_dim : str, optional
        An optional string allowing you to override the xarray dimension 
        used for x coordinates. Defaults to 'x'. Useful, for example, 
        if x and y dims instead called 'lat' and 'lon'.   
    y_dim : str, optional
        An optional string allowing you to override the xarray dimension 
        used for y coordinates. Defaults to 'y'. Useful, for example, 
        if x and y dims instead called 'lat' and 'lon'.
    export_tiff: str, optional
        If a filepath is provided (e.g 'output/output.tif'), will export a
        geotiff file. A named array is required for this operation, if one
        is not supplied by the user a default name, 'data', is used
    verbose : bool, optional
        Print debugging messages. Default False.
    **rasterio_kwargs : 
        A set of keyword arguments to rasterio.features.rasterize
        Can include: 'all_touched', 'merge_alg', 'dtype'.
    
    Returns
    -------
    xarr : xarray.DataArray
    
    """
    from rasterio.features import rasterize
    import xarray as xr

    # Check for a crs object
    try:
        crs = da.geobox.crs
    except:
        try:
            crs = da.crs
        except:
            if crs is None:
                raise ValueError("Please add a `crs` attribute to the "
                                 "xarray.DataArray, or provide a CRS using the "
                                 "function's `crs` parameter (e.g. crs='EPSG:3577')")
    
    # Check if transform is provided as a xarray.DataArray method.
    # If not, require supplied Affine
    if transform is None:
        try:
            # First, try to take transform info from geobox
            transform = da.geobox.transform
        # If no geobox
        except:
            try:
                # Try getting transform from 'transform' attribute
                transform = da.transform
            except:
                # If neither of those options work, raise an exception telling the 
                # user to provide a transform
                raise TypeError("Please provide an Affine transform object using the "
                                "`transform` parameter (e.g. `from affine import "
                                "Affine; Affine(30.0, 0.0, 548040.0, 0.0, -30.0, "
                                "6886890.0)`")
    
    # Grab the 2D dims (not time)    
    try:
        dims = da.geobox.dims
    except:
        dims = y_dim, x_dim  
    
    # Coords
    #xy_coords = [da[dims[0]], da[dims[1]]]
    xy_coords = [da['y'], da['x']]
    
    # Shape
    try:
        y, x = da.geobox.shape
    except:
        y, x = len(xy_coords[0]), len(xy_coords[1])
    
    # Reproject shapefile to match CRS of raster
    if verbose:
        print(f'Rasterizing to match xarray.DataArray dimensions ({y}, {x})')
    
    try:
        gdf_reproj = gdf.to_crs(crs=crs)
    except:
        # Sometimes the crs can be a datacube utils CRS object
        # so convert to string before reprojecting
        gdf_reproj = gdf.to_crs(crs={'init': str(crs)})
    
    # If an attribute column is specified, rasterise using vector 
    # attribute values. Otherwise, rasterise into a boolean array
    if attribute_col:        
        # Use the geometry and attributes from `gdf` to create an iterable
        shapes = zip(gdf_reproj.geometry, gdf_reproj[attribute_col])
    else:
        # Use geometry directly (will produce a boolean numpy array)
        shapes = gdf_reproj.geometry

    # Rasterise shapes into an array
    arr = rasterize(shapes=shapes,
                                      out_shape=(y, x),
                                      transform=transform,
                                      **rasterio_kwargs)
        
    # Convert result to a xarray.DataArray
    xarr = xr.DataArray(arr,
                        coords=xy_coords,
                        dims=dims,
                        attrs=da.attrs,
                        name=name if name else None)
    
    # Add back crs if xarr.attrs doesn't have it
    if xarr.geobox is None:
        xarr = assign_crs(xarr, str(crs))
    
    if export_tiff: 
        if verbose:
            print(f"Exporting GeoTIFF to {export_tiff}")
        write_cog(xarr,
                  export_tiff,
                  overwrite=True)
                
    return xarr

In [60]:
datar = datar.rename({'x':'longitude','y':'latitude'})

In [61]:
datar

<xarray.Dataset>
Dimensions:        (longitude: 8, latitude: 6, time: 25)
Coordinates:
  * longitude      (longitude) float64 -49.27 -49.27 -49.27 ... -49.25 -49.24
  * latitude       (latitude) float64 -21.91 -21.92 -21.92 -21.93 -21.93 -21.94
  * time           (time) datetime64[ns] 2023-03-30 2023-04-07 ... 2023-10-08
    spatial_ref    int64 0
Data variables:
    LST_Day_1km    (time, latitude, longitude) uint16 15141 15141 ... 15438
    LST_Night_1km  (time, latitude, longitude) uint16 14686 14686 ... 14655
    QC_Day         (time, latitude, longitude) uint8 0 0 0 0 0 0 ... 0 0 0 0 0 0
    QC_Night       (time, latitude, longitude) uint8 0 0 0 0 0 0 ... 0 0 0 0 0 0

In [62]:
mask = xr_rasterize(field,datar,
                  #  x_dim='x',
                  #  y_dim='y'
                   #export_tiff='masked2.tiff',
                   ) 

NameError: name 'assign_crs' is not defined

In [ ]:
mask = xr_rasterize(field,data,
                    # x_dim='x',
                    # y_dim='y',
                   #export_tiff='masked2.tiff',
                   ) 

# #mask data
datam = data.where(mask)

print(humanbytes(datam.nbytes))
#datam

In [ ]:
datam

In [ ]:
datam['QC_Day'] = xr.where(datam['QC_Day'] > 1, False, True)
datam['QC_Night'] = xr.where(datam['QC_Night'] > 1, False, True)

In [ ]:
datam['LST_Day_1km'] = datam['LST_Day_1km'] * 0.02 - 273.15
datam['LST_Night_1km'] = datam['LST_Night_1km'] * 0.02 - 273.15

datam['LST_Day_1km'] = datam['LST_Day_1km'] * datam['QC_Day'] 
datam['LST_Night_1km'] = datam['LST_Night_1km'] * datam['QC_Night'] 

print(datam['LST_Day_1km'].quantile([.05,.1,.5,.75,.99]))
print(datam['LST_Night_1km'].quantile([.05,.1,.5,.75,.99]))

In [ ]:
%%time

datam.compute()

In [ ]:
datam2 = datam.copy()

In [ ]:
# FOR LST
datam2['LST_Day_1km'] = xr.where((datam["LST_Day_1km"]> 350) | (datam["LST_Day_1km"]< 200), np.nan, datam["LST_Day_1km"])
datam2['LST_Night_1km'] = xr.where((datam["LST_Night_1km"]> 350) | (datam["LST_Night_1km"]< 200), np.nan, datam["LST_Night_1km"])

print(datam2['LST_Day_1km'].quantile([.05,.1,.5,.75,.99]))
print(datam2['LST_Night_1km'].quantile([.05,.1,.5,.75,.99]))


humanbytes(datam2.nbytes)

### z-score?

In [ ]:
# Group the time-series by month
#grouped_ds = datam2.groupby('time.month')
grouped_ds = data.groupby('time.month') # unmasked # unfiltered


# Calculate the mean and standard deviation for each month
mean = grouped_ds.mean('time')
std = grouped_ds.std('time')

# # Calculate the z-score
zscore = (datam2 - mean) / std

# # Print the z-score
zscore 

In [ ]:
# Define a function to calculate the z-score
def calculate_zscore(da):
    mean = da.mean()
    std = da.std()
    zscore = (da - mean) / std
    return zscore

# Apply the function to each month
zscore = grouped_ds.apply(calculate_zscore)
zscore

In [ ]:
print(zscore['LST_Day_1km'].quantile([.01,.1,.5,.75,.9,.99]))
print(zscore['LST_Night_1km'].quantile([.01,.1,.5,.75,.9,.99]))

In [ ]:
zscore

In [ ]:
# PLOT Z-SCORES

z2plot = zscore.isel(time=[x for x in range(450,481)])
var = 'LST_Day_1km'

g = z2plot[var].plot.imshow(
    cmap="RdBu_r", col="time", vmin = -3, vmax = 3, col_wrap=3, size=4
)
datetimes = z2plot[var].time.to_pandas().dt.strftime("%D")
print(datetimes)
for ax, datetime in zip(g.axes.flat, datetimes):
    ax.set_title(datetime)

In [ ]:
datam = datam.isel(time=list(range(0,len(datam),10)))
g = datam.plot.imshow(
    cmap="RdBu", col="time", robust=True, col_wrap=3, size=4
)
datetimes = datamt_.time.to_pandas().dt.strftime("%D")
print(datetimes)
for ax, datetime in zip(g.axes.flat, datetimes):
    ax.set_title(datetime)

In [ ]:
datamt_ = datamt.isel(time=list(range(0,len(datamt),10)))
g = datamt_.plot.imshow(
    cmap="RdBu", col="time", robust=True, col_wrap=3, size=4
)
datetimes = datamt_.time.to_pandas().dt.strftime("%D")
print(datetimes)
for ax, datetime in zip(g.axes.flat, datetimes):
    ax.set_title(datetime)

In [ ]:
gif(datamt, fps=3, cmap="RdBu")

# PAREI AQUI, MODIS ABRE

In [ ]:
ds_ = data.to_dataset(dim='band')

In [ ]:
rgb(ds_, bands = ["sur_refl_b01", "sur_refl_b04", "sur_refl_b03"], col='time')

### Mask dataset com fazenda

In [ ]:
%%time
#create mask versao dataarray
# which also helps to reduce data size
mask = xr_rasterize(field,data,
                    # x_dim='x',
                    # y_dim='y',
                   #export_tiff='masked2.tiff',
                   ) 

# #mask data
data = data.where(mask)

# #convert to float 32 to conserve memory
data = data.astype(np.float32)
data

In [ ]:
# calcula indices
ds_ = data.to_dataset(dim='band')

# os indices
indices = ["LAI", "EVI","NDCI","BSI"]
ds = calculate_indices(ds_, 
                       index= indices, 
                       satellite_mission='s2', 
                       drop=True);

In [ ]:
%%time
ds.compute()

In [ ]:
field2 = field.to_crs(3857)

## Criar zonas

In [22]:
%%time
column = 'parte'
fm = xr_rasterize(field2,data,attribute_col=column,verbose=True)
fm = fm.chunk(256)
fm.astype('uint8')
# fm_f64 = fm.astype('float64')
# fm_u8 = fm.astype('uint8')

fm.plot()

KeyError: 'latitude'

In [ ]:
fm.values

### Calculate stats for IVs dataset

#### single image

In [ ]:
zscore['LST_Day_1km']

In [ ]:
fm

In [ ]:
%%time
allstats = pd.DataFrame(index=zscore['LST_Day_1km'].time.values)

t =  zscore['LST_Day_1km'].time.values[2]
print(t)

data_ = zscore['LST_Day_1km'].sel(time=t).squeeze()
outstats = zonal_stats(zones=fm, values=data_).compute()
data_.close()
print('computing')
outstats 

In [ ]:
zscore['LST_Day_1km'].sel(time=zscore['LST_Day_1km'].time.values[0]).squeeze()

#### for a series

In [ ]:
%%time
nameout = 'MOD11A2_uniguiri'
verbose = True
indices = ['LST_Day_1km','LST_Night_1km']
ds = zscore.copy()
for iv in indices:

    # get stats for the first dataframe
    data_ = ds[iv].sel(time=ds[iv].time.values[0]).squeeze()
    print('computing stats for the first date')
    outst = zonal_stats(zones=fm, values=data_).compute()
    outst['date'] = str(ds[iv].time.values[0])
    data_.close()

    # and through the loop
    for t in data.time.values[1:]:
        data_ = ds[iv].sel(time=t).squeeze()
        if verbose: print(f'computing stats for {t}')
        
        outst1 = zonal_stats(zones=fm, values=data_).compute()
        outst1['date'] = str(t)
        outst = pd.concat([outst,outst1])
        data_.close()
        del outst1

    outst.to_csv(f'/home/jovyan/PlanetaryComputerExamples/myout_csv/grasspace/{nameout}_{iv}.csv')
    print(f'{nameout}_{iv}.csv SAVED \n \n')
    del outst

### Calculare stats for single image

### Calculate stats for a series

In [ ]:
end = time.time()
print(f'{(end - start):.0f} seconds')